<img src="https://pages.cnpem.br/capsuladaciencia/wp-content/uploads/sites/155/2022/10/Ilum.png" alt="Ilum - Escola de Ciência" width="200"/>

**Redes Neurais e Algoritmos Genéticos 2025**

**Prof. Dr. Daniel R. Cassar**

Rafael Dalacorte Erdmann (24017)

## Fera Formidável 13: A liga ternária mais cara do mundo

**Objetivo:** Encontre uma liga de três elementos que tenha o maior custo possível. A liga ternária deve ser da forma $x A.y B.z C$ sendo que $x + y + z = 100 ~g$, $x ≥ 5 ~g$, $y ≥ 5 ~g$, $z ≥ 5 ~g$ e “$A$”, “$B$” e “$C$” são elementos químicos diferentes. Utilize o preço dado abaixo [1]. Considere que qualquer composto com $3$ elementos químicos é chamado de liga.

**Dica:** Pode ser interessante criar uma função que gere a população inicial para garantir que $x + y + z = 100 g$.

**Dica 2:** Pode ser interessante criar uma ou mais funções de cruzamento e mutação neste problema de forma que todas elas garantam que, ao final do processo do operador, os indivíduos mantenham a característica de ter $x + y + z = 100 g$.

In [1]:
# preço em dólares por quilograma
preco = {
    "H": 1.39,
    "He": 24,
    "Li": 85.6,
    "Be": 857,
    "B": 3.68,
    "C": 0.122,
    "N": 0.14,
    "O": 0.154,
    "F": 2.16,
    "Ne": 240,
    "Na": 3.43,
    "Mg": 2.32,
    "Al": 1.79,
    "Si": 1.7,
    "P": 2.69,
    "S": 0.0926,
    "Cl": 0.082,
    "Ar": 0.931,
    "K": 13.6,
    "Ca": 2.35,
    "Sc": 3460,
    "Ti": 11.7,
    "V": 385,
    "Cr": 9.4,
    "Mn": 1.82,
    "Fe": 0.424,
    "Co": 32.8,
    "Ni": 13.9,
    "Cu": 6,
    "Zn": 2.55,
    "Ga": 148,
    "Ge": 1010,
    "As": 1.31,
    "Se": 21.4,
    "Br": 4.39,
    "Kr": 290,
    "Rb": 15500,
    "Sr": 6.68,
    "Y": 31,
    "Nb": 85.6,
    "Mo": 40.1,
    "Tc": 100000,
    "Ru": 10600,
    "Rh": 147000,
    "Pd": 49500,
    "Ag": 521,
    "Cd": 2.73,
    "In": 167,
    "Sn": 18.7,
    "Sb": 5.79,
    "Te": 63.5,
    "I": 35,
    "Xe": 1800,
    "Cs": 61800,
    "Ba": 0.275,
    "La": 4.92,
    "Ce": 4.71,
    "Pr": 103,
    "Nd": 57.5,
    "Pm": 460000,
    "Sm": 13.9,
    "Eu": 31.4,
    "Gd": 28.6,
    "Tb": 658,
    "Dy": 307,
    "Ho": 57.1,
    "Er": 26.4,
    "Tm": 3000,
    "Yb": 17.1,
    "Lu": 643,
    "Hf": 900,
    "Ta": 312,
    "W": 35.3,
    "Re": 4150,
    "Os": 12000,
    "Ir": 56200,
    "Pt": 27800,
    "Hg": 30.2,
    "Tl": 4200,
    "Pb": 2,
    "Bi": 6.36,
    "Po": 49200000000000,
    "Ac": 29000000000000,
    "Th": 287,
    "Pa": 280000,
    "U": 101,
    "Np": 660000,
    "Pu": 6490000,
    "Am": 750000,
    "Cm": 160000000000,
    "Bk": 185000000000,
    "Cf": 185000000000,
}


### Introdução

Esse problema é categorizado como um problema de satisfação de restrições (ou *constraint satisfaction problem*, CSP), ou seja, é um problema onde estamos tentando maximizar um ganho (também poderia ser minimizar uma perda), porém nossas variáveis tem limitações [1]. As limitações nesse tipo de problema podem ser classificadas como duras (*hard*), como nesse caso em que para o indivíduo ser válido ele deve ter $3$ elementos que somem massa $100 ~g$ sem nenhum deles ter menos que $5 ~g$, ou como moles (*soft*), quando há penalização para indivíduos fora da limitação, mas eles não se tornam inválidos.

Nosso objetivo aqui, portanto, é maximizar a função de preço do material sem gerar indivíduos inválidos conforme as regras definidas no enunciado.



Antes de começarmos, vamos importar as funções que serão utilizadas:

In [2]:
from pprint import pprint
from functools import partial
from random import seed

from funcoes_feras import populacao_liga as cria_populacao
from funcoes_feras import funcao_objetivo_liga
from funcoes_feras import funcao_objetivo_pop_liga
funcao_objetivo = partial(funcao_objetivo_pop_liga, tabela=preco)
from funcoes_feras import selecao_torneio_max as funcao_selecao
from funcoes_feras import cruzamento_ponto_simples_liga as funcao_cruzamento
from funcoes_feras import mutacao_troca_liga as funcao_mutacao_1
from funcoes_feras import mutacao_simples_liga
funcao_mutacao_2 = partial(mutacao_simples_liga, elementos=preco)
from funcoes_feras import mutacao_distribuicao_valores_liga as funcao_mutacao_3

E definir os parâmetros para o algoritmo genético:

In [3]:
SEMENTE_ALEATORIA = 42
seed(SEMENTE_ALEATORIA)

TAMANHO_POPULACAO = 100
NUM_GERACOES = 200
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
TAMANHO_TORNEIO = 3

### População

Primeiro, precisamos de uma população de candidatos. Para modelar o problema, utilizei uma estrutura de dicionário: cada candidato é um dicionário que contém três chaves, os elementos, contendo um float, as massas, cada. Tendo em mente que a massa mínima de cada é de $5 ~g$ e a soma das massas deve ser $100 ~g$, o resto pode ser decidido aleatoriamente pelo `random.uniform`.

In [4]:
populacao = cria_populacao(TAMANHO_POPULACAO, preco)

pprint(populacao)

[{'Be': 21.588372456000513, 'P': 10.379835064413705, 'Po': 68.03179247958579},
 {'Ar': 67.95675829675244, 'Np': 15.02159538297036, 'Si': 17.021646320277192},
 {'B': 69.43676680503879, 'Ba': 23.030469542725232, 'Ir': 7.532763652235979},
 {'Be': 15.59081273300643, 'Dy': 52.705830349782104, 'Hg': 31.703356917211465},
 {'Cm': 23.73745287345922, 'Cs': 32.2163019786601, 'Lu': 44.04624514788068},
 {'Cm': 40.440468649959435, 'H': 40.92222384304772, 'Sc': 18.637307506992848},
 {'Ni': 12.883396687312572, 'Rh': 12.458438466106637, 'Si': 74.65816484658079},
 {'Hg': 16.963293528427705, 'Pd': 73.60590322832232, 'Se': 9.430803243249983},
 {'S': 27.956440854592522, 'Sn': 60.34554230873853, 'Yb': 11.698016836668945},
 {'Bi': 54.074932346824774, 'Cd': 15.613276770632211, 'Pb': 30.311790882543022},
 {'C': 70.71080896703882, 'Pa': 24.004126121809872, 'Zn': 5.285064911151309},
 {'Al': 44.01338065235101, 'Sn': 27.358863083291155, 'Zn': 28.62775626435783},
 {'Cd': 35.19800951933932, 'In': 23.075056954039212,

### Função objetivo

Em seguida, deve-se montar a função objetivo, que nesse caso pega a quantidade do elemento (em $g$) e multiplica pelo preço por $1000 ~g$ do respectivo elemento (conforme tabela `preco`, do enunciado), resultando no custo. A soma dos custos dos três elementos é o fitness do problema, que deve ser maximizado pela `funcao_torneio_max`.

### Cruzamento

Por causa da estrutura de dados diferenciada, foi necessário preparar uma nova função de cruzamento. Para isso, utilizei um cruzamento de ponto simples que troca apenas os elementos (chaves do dicionário) do genoma herdado, de modo a evitar que a massa dos elementos quebre a delimitação de $100 ~g$. Com isso, os filhos possuem elementos de ambos os pais, mas enquanto um filho fica com as massas de um parente, o outro filho fica com as massas do outro.

### Mutação

Como mutação, pensei em três, que podem ocorrer independentemente em cada individuo:
- `mutacao_troca_liga`: troca a informação de massa entre dois elementos do candidato (objetivo: encontrar elementos mais adequados para cada valor), ex: {"Au":75, "Ni":20, "Pm":5} -> {"Au":20, "Ni":75, "Pm":5}
- `mutacao_simples_liga`: troca um elemento por outro possível (objetivo: trazer novos elementos), ex: {"Au":75, "Ni":20, "Pm":5} -> {"C":75, "Ni":20, "Pm":5}
- `mutacao_distribuicao_valores_liga`: escolhe dois elementos, soma suas massas e as redistribui aleatoriamente (objetivo: gerar novos valores), ex: {"Au":75, "Ni":20, "Pm":5} -> {"Au":45, "Ni":50, "Pm":5}

### Aplicação dos operadores

As gerações do algoritmo seguem o *pipeline*:

In [5]:
hall_da_fama = []

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao_1(proxima_geracao, CHANCE_DE_MUTACAO)
    funcao_mutacao_2(proxima_geracao, CHANCE_DE_MUTACAO)
    funcao_mutacao_3(proxima_geracao, CHANCE_DE_MUTACAO)
    
    # Atualização do hall da fama
    fitness = funcao_objetivo(proxima_geracao)
        
    maior_fitness = max(fitness)
    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    
    
    # Encerramento
    populacao = proxima_geracao

E por fim podemos exibir nosso melhor candidato.

In [6]:
fitness = funcao_objetivo(hall_da_fama)
maior_fitness = max(fitness)
indice = fitness.index(maior_fitness)
melhor_individuo_observado = hall_da_fama[indice]
custo = funcao_objetivo_liga(melhor_individuo_observado, preco)

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_observado, "com preço:", custo/10**12, "10**12 USD")
print()


Melhor individuo obtido por algoritmos genéticos:
{'Po': 88.86103695183586, 'Ac': 6.138646726199814, 'Bk': 5.000316321964348} com preço: 4.550908831609682 10**12 USD



Esse resultado concorda com nosso senso ao observar a tabela de preços: se queremos a liga mais cara e sabemos que alguns elementos (Po, Ac, Cf e Bk) são muito mais caros que outros, é intuitivo concluir que o melhor candidato terá a maior porcentagem possível do elemento mais caro, seguido das menores porcentagens possíveis dos próximos dois. Nesse caso, Cf e Bk tem o mesmo preço por quilo, de maneira que vez o algoritmo genético encontra a solução com um, vez com outro.

Com isso, o algoritmo rapidamente localiza quais elementos são os mais caros. Não é tão rápido para encontrar os valores exatos da massa de cada, pois o espaço de busca é muito maior, visto que suas tentativas e erros estão acontecendo na escala dos números float, diferentemente da lista de $92$ elementos. Vamos ver o erro relativo ao valor que intuitivamente identificamos:

In [7]:
custo_real = (preco["Po"]*90+preco["Ac"]*5+preco["Bk"]*5)/1000

erro_relativo = abs(custo-custo_real)/custo_real

print(f"Erro relativo da massa molar: {erro_relativo:0.4%}")

Erro relativo da massa molar: 0.5032%


Ou seja, o erro é menor que $1$%, de modo que a solução encontrada funciona bem, mesmo não sendo perfeita. Em problemas reais, pode não ser tão simples identificar a solução perfeita sem testar todas as possibilidades com busca exaustiva (o que muitas vezes se torna computacionalmente impossível).

### Conclusão

Com esse desenvolvimento, pudemos encontrar uma resposta suficiente para o problema, sem gerar indivíduos inválidos. Observe, ainda, que o resultado encontrado foi altamente eficiente, visto que não seria computacionalmente viável (ou, ao menos, tão rápido quanto foi com algoritmo genético) realizar uma busca exaustiva nesse problema para encontrar o candidato perfeito, visto que devem ser escolhidos combinações de três dentre $92$ elementos com massas variando entre $5$ e $90 ~g$.

### Referências

[1] EYAL WIRSANSKY. Hands-On Genetic Algorithms with Python: Applying genetic algorithms to solve real-world deep learning and artificial intelligence problems, 2020. Chapter 5: Constraint Satisfaction.